In [53]:
# necessary imports
import numpy as np
import os, random, shutil
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import torchvision

import torchvision.transforms as transforms
import pandas as pd
from torchvision.io import read_image
import scipy.io as sio

import csv
import zipfile
from math import floor, log10

In [22]:
# use GPU if possible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'using {device}')


using cuda:0


In [61]:
#checks if a labels.csv exists, if so, this code doesn't run. 
if(not os.path.exists('labels.csv')):
    #takes in image gps coordinates and creates labels of which city picture is from
    mat = sio.loadmat('GPS_Long_Lat_Compass.mat')
    mat =mat['GPS_Compass']
    print(type(mat))
    labels_arr = np.zeros(len(mat))
    #0 is pitsbourgh, 1 is florida, 2 is new york
    for i in range(len(mat)):
        if(mat[i,0]>40):
            if(mat[i,1]<-79):
                labels_arr[i]=0
            else:
                labels_arr[i]=2
        else:
            labels_arr[i]=1

    #creates list of image names for labels.csv file. (this will make loading data into pytorch easier)
    label_list=[]
    for i, city in enumerate(labels_arr):
        for j in range(1,6):
            label_list.append([f'{i+1:06}_{j}.jpg',int(city)])


        
    with open('labels.csv', 'w',newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(label_list)
        

<class 'numpy.ndarray'>


In [ ]:
archive = zipfile.ZipFile(zip_filename, 'r')
image = archive.read(filename)

In [ ]:
#class for pytorch to build dataset
class GeoLocationData(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = labels
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #change 
        image.to(device, dtype=torch.float32)
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [25]:
class geonet(nn.Module):
    def __init__(self, model_layers):
        super().__init__()
       
        # create the BiT model and freeze all layers except the last two:
        bit_model = timm.create_model('resnetv2_50x1_bitm', pretrained=True, num_classes=model_layers[0])
        for param in list(bit_model.parameters())[:-2]:
            param.requires_grad = False
        model_layers=0
        layers = [bit_model]
        for i in range(len(model_layers)-1):
            layers.append(nn.Linear(model_layers[i],model_layers[i+1]))
            if(i!=len(model_layers)-1):
                layers.append(nn.ReLU())
            
        self.network=nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        return self.network(x)
        

In [27]:
#model layers decreasing to picking one of cities in dataset
model = geonet([1024, 512, 256, 3])

learning_rate = 1e-4
batch_size = 128
epochs = 15



loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def training(data, model, loss, optimizer):
    size = len(data.dataset)
    for batch, (X, y) in enumerate(data):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



geonet(
  (network): Sequential(
    (0): ResNetV2(
      (stem): Sequential(
        (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0.0)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (stages): Sequential(
        (0): ResNetStage(
          (blocks): Sequential(
            (0): PreActBottleneck(
              (downsample): DownsampleConv(
                (conv): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (norm): Identity()
              )
              (norm1): GroupNormAct(
                32, 64, eps=1e-05, affine=True
                (drop): Identity()
                (act): ReLU(inplace=True)
              )
              (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm2): GroupNormAct(
                32, 64, eps=1e-05, affine=True
          